In [1]:
import mlflow
import mlflow.spark as sparkm
import time
import os
is_databricks  = os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None
is_databricks
print("MLflow Version:",mlflow.version.VERSION)
mlflow.tracking.get_tracking_uri()

if is_databricks:
    client = mlflow.tracking.MlflowClient()
    naptime = int(2)
    data_path = "/dbfs/ml/loan_stats.csv"
    model_name = "bk-loan-clf"
    experiment_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
else:
    data_path = "../data/loan_stats.csv"
    mlflow.set_tracking_uri("http://localhost:5000")
    client = mlflow.tracking.MlflowClient()
    naptime = 0
    model_name = "bk-loan-clf"
    experiment_name = "bk-loan-clf-exp"
    mlflow.set_experiment(experiment_name)


experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

print("experiment_id:",experiment_id)
print("experiment_name:", experiment_name)

runs = client.list_run_infos(experiment_id)
print("#runs:",len(runs))
for info in runs:
    print(info.run_id)

MLflow Version: 1.5.0
experiment_id: 4153807906280208
experiment_name: /Users/bhavin.kukadia@databricks.com/Workshops/AllinOne/_helper_azureml
#runs: 0

In [2]:
if is_databricks:
    host_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("browserHostName").get()
    uri = "https://{}/#mlflow/experiments/{}".format(host_name,experiment_id)
    displayHTML("""<b>Experiment URI:</b> <a href="{}">{}</a>""".format(uri,uri))

Experiment URI: https://eastus2.azuredatabricks.net/#mlflow/experiments/4153807906280208

In [3]:
if is_databricks:
    uri = "https://{}/#mlflow/models/{}".format(host_name,model_name)
    displayHTML("""<b>Registered Model URI:</b> <a href="{}">{}</a>""".format(uri,uri))

Registered Model URI: https://eastus2.azuredatabricks.net/#mlflow/models/bk-loan-clf

In [4]:
def show_versions(versions):
    for v in versions:
        print(f"{v.run_id} {v.version} {v.current_stage} '{v.description}'")

In [5]:
model_uri = f"models:/{model_name}/production"
model_uri
model = mlflow.spark.load_model(model_uri)
model

--------------------------------------------------------------------------- 
 MlflowException Traceback (most recent call last)
 <command-4153807906280220> in <module> 
 1 model_uri = f"models:/{model_name}/production" 
 2 model_uri
 ----> 3 model = mlflow . spark . load_model ( model_uri ) 
 4 model

 /databricks/python/lib/python3.7/site-packages/mlflow/spark.py in load_model (model_uri, dfs_tmpdir) 
 429 elif ModelsArtifactRepository . is_models_uri ( model_uri ) : 
 430 runs_uri = model_uri
 --> 431 model_uri = ModelsArtifactRepository . get_underlying_uri ( model_uri ) 
 432 _logger . info ( "'%s' resolved as '%s'" , runs_uri , model_uri ) 
 433 flavor_conf = _get_flavor_configuration_from_uri ( model_uri , FLAVOR_NAME ) 

 /databricks/python/lib/python3.7/site-packages/mlflow/store/artifact/models_artifact_repo.py in get_underlying_uri (uri) 
 65 if len ( latest ) == 0 : 
 66 raise MlflowException("No versions of model with name '{name}' and "
 ---> 67 "stage '{stage}' found".format(name=name, stage=stage))
 68 version = latest [ 0 ] . version
 69 return client . get_model_version_download_uri ( name , version ) 

 MlflowException : No versions of model with name 'bk-loan-clf' and stage 'production' found